In [ ]:
import nltk
nltk.download('all')

## Reading File and creating DataFrame

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pickle

In [ ]:
import pandas as pd
wiki_files = pd.read_csv('wiki_sample.csv')
wiki_dataframe = pd.DataFrame(wiki_files)
wiki_dataframe

In [ ]:
sum = 0
for entry in wiki_dataframe['tokenized_content']:
    sum += len(entry)
    

In [ ]:
sum
len(vocab)

In [ ]:
total_words
len(inv_idx)

## Preprocessing and Creating Vocab

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pycountry
lemmatizer = WordNetLemmatizer()
from names_dataset import NameDatasetV1 # v1
names = NameDatasetV1()

stop_words = set(stopwords.words('english'))
# for domain_stop_word in domain_stop_words:
#     stop_words.add(domain_stop_word)

In [ ]:
import pandas as pd
companies_file = pd.read_csv('companies_sorted.csv')
companies_dataframe = pd.DataFrame(companies_file)
companies_dataframe

In [ ]:
companies = set(companies_dataframe['name'])

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
words = set(nltk.corpus.words.words())
lowerCasedWords = map(lambda word: word.lower(), nltk.corpus.words.words())
lowerCasedWords = set(list(lowerCasedWords))
for country in list(pycountry.countries):
    lowerCasedWords.add(country.name.lower())
for company in companies:
    lowerCasedWords.add(str(company).lower())

In [ ]:
# lemmatization, lowercase, remove non alphanumeric, remove non-english, remove numbers and stopword removal
rejected_content = []
def preprocess_vocab(row, lenTitle):
    filtered_content = []
    for token in nltk.word_tokenize(row['content'][lenTitle:]):
        token = lemmatizer.lemmatize(token).lower()
        if names.search_first_name(token) or names.search_last_name(token) or ((token in lowerCasedWords) and (token not in stop_words) and (token.isalpha())):
            filtered_content.append(token) 
        else:
            rejected_content.append(token)
        
    return filtered_content

In [ ]:
wiki_dataframe['tokenized_content'] = wiki_dataframe.progress_apply(lambda row: preprocess_vocab(row, len(row['title'])), axis=1)

In [ ]:
wiki_dataframe.to_pickle('./wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl')

In [4]:
import pickle5 as pickle
with open("./data/wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl", "rb") as pickle_file:
    wiki_dataframe = pickle.load(pickle_file)
# wiki_dataframe = pd.read_pickle('./wiki_dataframe_augmented_nltk_corpus_to_remove_non-english.pkl')

In [ ]:
wiki_dataframe

## Creating Part of Inverted Index

In [ ]:
from collections import defaultdict
from collections import Counter

inv_idx = defaultdict(list)
vocab = set()
heaps_law_dataset = list()

most_freq = []
id = 1
total_words = 0
for document in tqdm(wiki_dataframe['tokenized_content']):
    counter = Counter(document)
    most_occur = counter.most_common(1)
    most_freq.append(most_occur)
    heaps_law_dataset.append((total_words, len(vocab)))
    for word in document:
        inv_idx[word].append(id)
        total_words += 1
        vocab.add(word)
    id +=1

In [ ]:
wiki_dataframe['most_frequent_term'] = most_freq

In [ ]:
wiki_dataframe

In [ ]:
from collections import OrderedDict
inv_idx_ordered = OrderedDict(sorted(inv_idx.items(), key=lambda item: len(item[1]), reverse=True))

In [ ]:
words = list(inv_idx_ordered.keys())
domain_stop_words = words[0:10000]

## Heaps Law

In [ ]:
heaps_law_dataset[len(heaps_law_dataset)-1]

In [ ]:
import matplotlib.pyplot as plt
import math

def heaps_law(list_to_graph):
        x = list()
        y = list()
        
        for item in list_to_graph:
            x.append(item[0])
            y.append(item[1])

        plt.plot(x, y)
        plt.xlim(1, x[-1])
        plt.ylim(1, y[-1])
        plt.savefig("heaps_law_words_from_nltk_english_corpus.png")

In [ ]:
heaps_law(heaps_law_dataset)

## Zipfs Law

In [ ]:
import matplotlib.pyplot as plt
import math

def zipfs_law(list_to_graph):
        x = list()
        y = list()
        
        for i, word in enumerate(list_to_graph):
            x.append(i+1)
            y.append(len(inv_idx_ordered[word]))
            

        plt.loglog(x, y)
        plt.savefig("zipfs_law.png")

In [ ]:
zipfs_law(inv_idx_ordered)

## Finishing Inverted Index

In [ ]:
for value in tqdm(inv_idx_ordered.items()):
    inv_idx[value[0]] = (Counter(value[1]))

In [ ]:
inv_idx

In [ ]:
file_to_write = open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "wb")
pickle.dump(inv_idx, file_to_write)

In [5]:
inv_idx = pickle.load(open("inv_idx_augmented_nltk_corpus_to_remove_non-english.pkl", "rb"))

## Suggesting Queries

In [ ]:
from os import listdir
from os.path import isfile, join
aol_query_log = pd.read_csv('project_1_AOL_query_log/Clean-Data-01.txt', sep="\t")
for file in listdir('project_1_AOL_query_log')[1:]:
    aol_query_log = aol_query_log.append(pd.read_csv('project_1_AOL_query_log/' + str(file), sep="\t"), ignore_index=True)

In [ ]:
# candidate_scores = []
# for candidate in candidates:
#     num_sessions_q_modified = 0
#     for ID in set(aol_query_log[aol_query_log['Query'] == original_query]['AnonID']):
#         candidate_list = list(aol_query_log[aol_query_log['AnonID'] == ID]['Query'])
#         try:
#             index = candidate_list.index(original_query)
#         except: 
#             continue
#         if candidate[1] in candidate_list[index+1:]:
#             num_sessions_q_modified += 1
#         candidate_scorse.append((candidate, num_sessions_q_modified / num_session_containing_q))

In [ ]:
set(aol_query_log[aol_query_log['Query'] == 'vietnam']['AnonID'])


In [ ]:
aol_query_log

In [ ]:
type(aol_query_log['AnonID'][0])

In [ ]:
# lemmatization, lowercase, remove non alphanumeric, and stopword removal
def query_logs_preprocessing(row):
    filtered_content = ""
    for token in nltk.word_tokenize(str(row['Query'])):
        token = lemmatizer.lemmatize(token).lower()
        if token not in stop_words and token.isalpha():
            filtered_content += token + " "
    return filtered_content[:-1]

In [ ]:
aol_query_log['Processed Query'] = aol_query_log.progress_apply(lambda row: query_logs_preprocessing(row), axis=1)
aol_query_log.to_pickle('./data/aol_query_log_data.pkl')
aol_query_log

In [ ]:
aol_queries = aol_query_log['Processed Query'].values
def identify_candidate_queries(query):
    candidate_queries = set()
    for aol_query in aol_queries:
        if len(aol_query.split()) > len(query.split()):
            if aol_query.startswith(query):
                candidate_queries.add(aol_query)
    return candidate_queries

#### Ranking Candidate Suggestions
##### 𝑆𝑐𝑜𝑟𝑒(𝐶𝑄, 𝑞′) = \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑖𝑠 𝑚𝑜𝑑𝑖𝑓𝑖𝑒𝑑 𝑡𝑜 𝐶𝑄 ÷ \# 𝑜𝑓 𝑠𝑒𝑠𝑠𝑖𝑜𝑛𝑠 𝑖𝑛 𝑤ℎ𝑖𝑐ℎ 𝑞 𝑎𝑝𝑝𝑒𝑎𝑟𝑠

In [ ]:
set(aol_query_log[aol_query_log['AnonID'] == 217]['Query'])

In [ ]:
aol_queries = aol_query_log['Query'].values
# n^3 algorithm ask dr. pera if this is valid algorithm
def OLD_rank_candidate_queries(original_query, candidates):
    candidate_scores = []
    num_session_containing_q = 0
    num_sessions_q_modified = 0
    
    sessions_with_original_query = set(aol_query_log[aol_query_log['Query'] == original_query]['AnonID'])
    num_session_containing_q = len(sessions_with_original_query)
            
    for candidate in tqdm(candidates):
        num_sessions_q_modified = 0
        for ID in sessions_with_original_query:
            candidate_list = list(aol_query_log[aol_query_log['AnonID'] == ID]['Query'])
            try:
                index = candidate_list.index(original_query)
            except: 
                continue
            if candidate[1] in candidate_list[index+1:]:
                num_sessions_q_modified += 1
        candidate_scores.append((candidate, num_sessions_q_modified / num_session_containing_q))
    return sorted(candidate_scores, key=lambda candidate_score: candidate_score[1], reverse=True)

In [ ]:
def rank_candidate_queries(original_query, candidates):
    candidate_scores = []
    num_sessions_containing_q = 0    
    sessions_with_original_query = set(aol_query_log[aol_query_log['Query'] == original_query]['AnonID'])
    # print(sessions_with_original_query)
    # return sessions_with_original_query 
    num_sessions_containing_q = len(sessions_with_original_query)
    
    for candidate in tqdm(candidates): #cut off at certain number of candidates?
        sessions_with_candidate_query = set(aol_query_log[aol_query_log['Query'] == candidate]['AnonID'])
        sessions_with_both_queries = sessions_with_original_query & sessions_with_candidate_query
        if num_sessions_containing_q != 0:
            candidate_scores.append((candidate, len(sessions_with_both_queries) / num_sessions_containing_q))
        
    for candidate in candidates: # supplementing the final list if the ranked results dont equal five 
        if len(candidate_scores) >= 5:
            break
        candidate_scores.append((candidate, 0)) # candidate score of 0 is default rank
    return sorted(candidate_scores, key=lambda candidate_score: candidate_score[1], reverse=True)

In [ ]:
def find_rank_candidate_queries(query):
    candidate_list = identify_candidate_queries(query)
    return rank_candidate_queries(query, candidate_list)

In [ ]:
find_rank_candidate_queries("frozen")

## Relevance Ranking

### Identifying Candidate Resources

In [6]:
from itertools import combinations
def identify_candidate_resources(query):
    results = set()
    split_query = query.split()
    n = len(split_query)
    candidate_list = list()
    for term in split_query:
        if len(inv_idx[term]) > 0:
            candidate_list.append(set(inv_idx[term].keys()))
    if len(candidate_list) > 0:
        results = set.intersection(*candidate_list)
    if len(results) <= 50:
        for combination in combinations(split_query, n - 1):
            candidate_list = list()
            for term in combination:
                  candidate_list.append(inv_idx[term])
            if len(candidate_list) > 0:
                results = set.intersection(*candidate_list)
            if len(results) > 50:
                break
            else:
                n -= 1
    return results


In [7]:
identify_candidate_resources('sex')

{229377,
 327681,
 278529,
 507907,
 655365,
 737284,
 131080,
 720904,
 737288,
 442384,
 131095,
 229399,
 131097,
 901151,
 1179680,
 1277987,
 524325,
 426023,
 360490,
 1163306,
 458796,
 1163309,
 1163310,
 639024,
 589873,
 524338,
 442417,
 639026,
 426043,
 1163325,
 442430,
 999486,
 458817,
 1097795,
 1032262,
 1130568,
 81993,
 557130,
 655436,
 196685,
 196687,
 229456,
 655442,
 1146962,
 114770,
 278611,
 639059,
 770135,
 1130584,
 1081434,
 245851,
 32860,
 1163358,
 852065,
 1130594,
 737379,
 671846,
 639083,
 573548,
 933995,
 933998,
 442482,
 737394,
 98420,
 999539,
 1065078,
 999543,
 1032311,
 491642,
 163963,
 721020,
 131197,
 32894,
 999547,
 999549,
 999550,
 999551,
 98435,
 1097857,
 180356,
 1048710,
 196743,
 999562,
 163979,
 1032335,
 180369,
 49301,
 704664,
 213145,
 835741,
 1147039,
 1147042,
 508066,
 737443,
 65705,
 327849,
 950443,
 180394,
 966826,
 737453,
 508082,
 1065141,
 245942,
 295095,
 1097911,
 901307,
 786620,
 442557,
 295103,
 57

### TF-IDF
#### 𝑇𝐹(𝑤, 𝑑) = 𝑓𝑟𝑒𝑞(𝑤, 𝑑) ÷ (𝑚𝑎𝑥_𝑑)
#### 𝐼𝐷𝐹(𝑤) = 𝑙𝑜𝑔__2 (𝑁 ÷ 𝑛_𝑤)

In [ ]:
len(wiki_dataframe)

In [11]:
import math
def tf_idf(split_query, document_id):
    score = 0
    for term in split_query:
        # print("doc ", document_id)
        # print("inv index term:",inv_idx[term][document_id])
        # print("most freq:",wiki_dataframe['most_frequent_term'][document_id - 1][0][1])
        score += (inv_idx[term][document_id] / wiki_dataframe['most_frequent_term'][document_id - 1][0][1]) * math.log((len(wiki_dataframe) / len(inv_idx[term])), 2)
    return score

### Ranking candidate resources

In [9]:
def rank_candidate_resources(query, candidate_resources):
  ranked_candidates = {}
  for document_id in candidate_resources:
    ranked_candidates[document_id] = tf_idf(query.split(), document_id)
  return sorted(ranked_candidates.items(), key=lambda item: item[1], reverse=True)

In [12]:
def find_and_rank_candidate_resources(query):
    candidates = identify_candidate_resources(query)
    return rank_candidate_resources(query, candidates)
relevant_resources = find_and_rank_candidate_resources("frozen")
relevant_resources

[(471159, 9.534867165939293),
 (798969, 9.534867165939293),
 (561640, 9.534867165939293),
 (791185, 9.534867165939293),
 (947248, 9.534867165939293),
 (1283442, 9.534867165939293),
 (558466, 9.534867165939293),
 (1230366, 9.534867165939293),
 (784210, 9.534867165939293),
 (596022, 9.534867165939293),
 (665915, 9.534867165939293),
 (768525, 9.534867165939293),
 (789073, 9.534867165939293),
 (432721, 9.534867165939293),
 (1026710, 9.534867165939293),
 (44219, 9.534867165939293),
 (466431, 9.534867165939293),
 (884349, 9.534867165939293),
 (376546, 9.534867165939293),
 (1089337, 9.534867165939293),
 (831304, 9.534867165939293),
 (593740, 9.534867165939293),
 (167806, 9.534867165939293),
 (976434, 8.172743285090823),
 (672356, 7.627893732751435),
 (908078, 7.627893732751435),
 (39731, 7.1511503744544695),
 (1150805, 7.1511503744544695),
 (1195639, 6.8106194042423525),
 (1178774, 6.674407016157504),
 (328779, 6.356578110626195),
 (349565, 6.356578110626195),
 (54889, 6.356578110626195),
 (2

### Snippet Generation

In [24]:
def get_snippet(query, document_id):
    snippet = tuple() # in the form (title, sentences)
    row = wiki_dataframe[wiki_dataframe['id'] == document_id]
    snippet[0] = row["title"]
    snippet[1] = generate_sentence_snippets(query, row["content"])
    return snippet

In [21]:
wiki_dataframe["title"][0]

'Morocco–Saudi Arabia relations'

In [23]:
def generate_sentence_snippets(query, content):
    top_two = list() # format is (sentence, cosine_similarity_score)
    vectorized_query = query ######### need to vectorize the query
    for sentence in content:
        vectorized_sentence = sentence ######### need to vectorize the sentence
        sentence_score = cosine_similarity(vectorized_query, vectorized_sentence)
        if len(top_two) < 2:
            top_two.append((sentence, sentence_score))
            top_two.sort(key=lambda item: item[1], reverse=True)
            continue
        
        for index, entry in enumerate(top_two): # this loop should only run twice
            if sentence_score > entry[index][1]: 
                top_two.insert(index, (sentence, sentence_score))
                continue
    
    return top_two[0][0] + "\n" + top_two[1][0]

In [ ]:
from scipy import spatial
def cosine_similarity(vectorized_query, vectorized_sentence):
    return (1 - spatial.distance.cosine(vectorized_sentence, vectorized_query))